In [132]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from collections import Counter
from datetime import datetime, timedelta
from sklearn import metrics
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn import ensemble
import sklearn.metrics
import time
from datetime import datetime

%matplotlib inline                                
#no need to do plt.show()

In [133]:
train = pd.read_csv('./train.csv',parse_dates=['date'])
test = pd.read_csv('./test.csv',parse_dates=['date'],index_col = 0)
print( ' train shape:{}, columns:{} '.format(train.shape,train.columns))
print( ' test shape:{}, columns:{}'.format(test.shape,test.columns))

print(train['date'].head())
print(train.columns)
train.head()

 train shape:(14006, 3), columns:Index(['id', 'date', 'speed'], dtype='object') 
 test shape:(3504, 1), columns:Index(['date'], dtype='object')
0   2017-01-01 00:00:00
1   2017-01-01 01:00:00
2   2017-01-01 02:00:00
3   2017-01-01 03:00:00
4   2017-01-01 04:00:00
Name: date, dtype: datetime64[ns]
Index(['id', 'date', 'speed'], dtype='object')


,id,date,speed
0,0,2017-01-01 00:00:00,43.002930
1,1,2017-01-01 01:00:00,46.118696
2,2,2017-01-01 02:00:00,44.294158
3,3,2017-01-01 03:00:00,41.067468
4,4,2017-01-01 04:00:00,46.448653


In [152]:
###处理特征
#计划   season4个特征+周几+年+月+日+时+是否节假日

#周几+年+月+日+时
for df in (train,test): 
    df['year']  = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day']   = df['date'].dt.day
    df['hr']    = df['date'].dt.hour
    #     df["weekdayth"] = df['date'].dt.weekday                                 #周几


#     df["weekofyear"] = df['date'].dt.isocalendar().week.astype("int")
#     df["dayofyear"] = df['date'].dt.dayofyear

######

         
            

    
for df in (train,test):
    df['year1'] = np.nan 
    
    df.loc[(df.year==2017),'year1']= 0 
    df.loc[(df.year==2018),'year1']= 1  
    

    
#加香港2017 2018节假日       ：date speed year month day hr weekday weekofyear season dayofyear hr-categori y/n-holiday
public_vacation_list = [
    '20180101', '20180216','20180217','20180219','20180330','20180331','20180402','20180405',
    '20180501','20180522','20180618','20180702','20180925','20181001','20181017','20181225','20181226',
    '20170102','20170128','20170130','20170131','20170404','20170414','20170415','20170417','20170501',
    '20170503','20170530','20170701','20171002','20171005','20171028','20171225','20171226'
] 
for df in (train,test):
    df['only-ymd'] = df['date'].apply(lambda x: x.strftime('%Y%m%d'))
    df['y/n-holiday'] = df['only-ymd'].apply(lambda x: 1 if x in public_vacation_list else 0)


    
    
    
df_train = train.drop(columns= ['only-ymd','date','id'], axis= 1).copy()
df_test = test.drop(columns= ['only-ymd','date'], axis= 1).copy()

df_train

,speed,year,month,day,hr,year1,y/n-holiday
0,43.002930,2017,1,1,0,0.0,0
1,46.118696,2017,1,1,1,0.0,0
2,44.294158,2017,1,1,2,0.0,0
3,41.067468,2017,1,1,3,0.0,0
4,46.448653,2017,1,1,4,0.0,0
...,...,...,...,...,...,...,...
14001,19.865269,2018,12,31,12,1.0,0
14002,17.820375,2018,12,31,15,1.0,0
14003,12.501851,2018,12,31,16,1.0,0
14004,15.979319,2018,12,31,18,1.0,0


In [144]:
day=df_train['day']
weekdayindex=[7]
dayindex=1
for i in range(1,len(day)):
    if int(day[i-1])==int(day[i]):
        weekdayindex.append(weekdayindex[-1])
    else:
        dayindex+=1
        if dayindex%7==2:
            weekdayindex.append(1)
                
        elif dayindex%7==3:
            weekdayindex.append(2)
                
        elif dayindex%7==4:
            weekdayindex.append(3)
                
        elif dayindex%7==5:
            weekdayindex.append(4)
        elif dayindex%7==6:
            weekdayindex.append(5)
        elif dayindex%7==0:
            weekdayindex.append(6)
        else: 
            weekdayindex.append(6)

print(len(weekdayindex))

df_train['weekdayindex']=weekdayindex  


14006


In [145]:
day=df_test['day']
weekdayindex=[1]
dayindex=1
for i in range(1,len(day)):
    if int(day[i-1])==int(day[i]):
        weekdayindex.append(weekdayindex[-1])
    else:
        dayindex+=1
        if dayindex%7==2:
            weekdayindex.append(2)
                
        elif dayindex%7==3:
            weekdayindex.append(3)
                
        elif dayindex%7==4:
            weekdayindex.append(4)
                
        elif dayindex%7==5:
            weekdayindex.append(5)
        elif dayindex%7==6:
            weekdayindex.append(6)
        elif dayindex%7==1:
            weekdayindex.append(1)
        else: 
            weekdayindex.append(7)

print(len(weekdayindex))

df_test['weekdayindex']=weekdayindex 

3504


In [146]:
df_train=df_train.drop(['year'],axis=1)
df_test=df_test.drop(['year'],axis=1)

In [147]:
df_train

,speed,month,day,hr,year1,y/n-holiday,weekdayindex
0,43.002930,1,1,0,0.0,0,7
1,46.118696,1,1,1,0.0,0,7
2,44.294158,1,1,2,0.0,0,7
3,41.067468,1,1,3,0.0,0,7
4,46.448653,1,1,4,0.0,0,7
...,...,...,...,...,...,...,...
14001,19.865269,12,31,12,1.0,0,3
14002,17.820375,12,31,15,1.0,0,3
14003,12.501851,12,31,16,1.0,0,3
14004,15.979319,12,31,18,1.0,0,3


In [148]:
df_test

,month,day,hr,year1,y/n-holiday,weekdayindex
id,,,,,,
0,1,1,2,1.0,1,1
1,1,1,5,1.0,1,1
2,1,1,7,1.0,1,1
3,1,1,8,1.0,1,1
4,1,1,10,1.0,1,1
...,...,...,...,...,...,...
3499,12,31,17,1.0,0,2
3500,12,31,19,1.0,0,2
3501,12,31,21,1.0,0,2


In [125]:
df_train.corr()['speed'] 

speed           1.000000
month          -0.028827
day            -0.019485
hr             -0.390044
weekdayth       0.115887
year1           0.015102
y/n-holiday     0.055386
weekdayindex   -0.003058
Name: speed, dtype: float64

In [120]:
####不需要编码了
from sklearn.model_selection import train_test_split
#留出法划分数据,就按照经典的37分
Train, Valid = train_test_split(df_train, test_size = 0.2)  


X_train = Train.drop(['speed'], axis=1)     #提取训练数据自变量

Y_train = Train["speed"]                    #提取训练数据因变量

X_valid = Valid.drop(['speed'], axis=1)       #提取测试数据自变量

Y_valid = Valid["speed"]                      #提取测试数据因变量


Y_valid = Y_valid.reset_index().drop('index',axis = 1)
Y_train = Y_train.reset_index().drop('index',axis = 1)
#重新进行索引顺序



#加载模型，放入相关数据
dtrain = xgb.DMatrix(X_train, label=Y_train)    #放入训练数据
dvalid = xgb.DMatrix(X_valid, label=Y_valid)      #放入测试拟合与否的数据

dtest = xgb.DMatrix(df_test)                #放入最终的需要提交的检测数据

In [21]:
# cv_params = {'colsample_bylevel':  [0.8,0.9,1]}
# other_params = {'learning_rate': 0.01, 'n_estimators': 1550, 'max_depth': 12, 'min_child_weight': 1, 'seed': 65,
#                     'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.2, 'reg_alpha': 3, 'reg_lambda': 4}

# model = xgb.XGBRegressor(**other_params)
# optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
# optimized_GBM.fit(X_train, Y_train)
# evalute_result = optimized_GBM.cv_results_['mean_test_score']
# print('每轮迭代运行结果:{0}'.format(evalute_result))
# print('参数的最佳取值：{0}'.format(optimized_GBM.cv_results_['params']))


# cv_params = {'reg_alpha': [0.05, 0.1, 1, 2, 3,4], 'reg_lambda': [0.05, 0.1, 1, 2, 3,4]}
# other_params = {'learning_rate': 0.1, 'n_estimators': 1050, 'max_depth': 50, 'min_child_weight': 0, 'seed': 0,
#                     'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.6, 'reg_alpha': 0, 'reg_lambda': 1}
# model = xgb.XGBRegressor(**other_params)
# optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
# optimized_GBM.fit(X_train, Y_train)
# evalute_result = optimized_GBM.cv_results_['mean_test_score']
# print('每轮迭代运行结果:{0}'.format(evalute_result))
# print('参数的最佳取值：{0}'.format(optimized_GBM.cv_results_['params']))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done 180 out of 180 | elapsed: 11.8min finished


每轮迭代运行结果:[-22.9646399  -21.97946383 -17.64374467 -16.98805113 -16.79100916
 -16.78302524 -22.74817243 -21.82861379 -17.63333697 -17.13497932
 -16.8360295  -16.66153928 -20.07708988 -19.70187171 -17.17994928
 -16.84345964 -16.7670804  -16.66837325 -18.59620671 -18.26669722
 -16.81799349 -16.77038763 -16.63674349 -16.65194442 -17.69955557
 -17.40029162 -16.78641168 -16.63740025 -16.64000186 -16.61306818
 -17.13046051 -17.05107074 -16.65767646 -16.63505121 -16.66142417
 -16.69845706]
参数的最佳取值：[{'reg_alpha': 0.05, 'reg_lambda': 0.05}, {'reg_alpha': 0.05, 'reg_lambda': 0.1}, {'reg_alpha': 0.05, 'reg_lambda': 1}, {'reg_alpha': 0.05, 'reg_lambda': 2}, {'reg_alpha': 0.05, 'reg_lambda': 3}, {'reg_alpha': 0.05, 'reg_lambda': 4}, {'reg_alpha': 0.1, 'reg_lambda': 0.05}, {'reg_alpha': 0.1, 'reg_lambda': 0.1}, {'reg_alpha': 0.1, 'reg_lambda': 1}, {'reg_alpha': 0.1, 'reg_lambda': 2}, {'reg_alpha': 0.1, 'reg_lambda': 3}, {'reg_alpha': 0.1, 'reg_lambda': 4}, {'reg_alpha': 1, 'reg_lambda': 0.05}, {'reg_a

In [123]:
xgb_pars = {
    'nthread':            4,  
    'booster':            'gbtree',
    'objective':          'reg:squarederror',
    'eta':                0.01,
    'min_child_weight':   0,   
    'max_depth':          70,  
    'subsample':          0.5,
    'colsample_bytree':   1,
    'gamma':              0,
    'lambda':             3,
    'seed':               1000,
    'silent':             0
}

xgb_par=list(xgb_pars.items())
model = xgb.train(xgb_par, dtrain, 1500)





[21:41:55] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [124]:
from sklearn import metrics
pred = model.predict(dvalid)
print('testmse', metrics.mean_squared_error(Y_valid,pred))

testmse 13.294856355111587


In [118]:
# pred = model.predict(dtest)
# result = pd.DataFrame({"id":range(3504), "speed":pred})
# result.to_csv("submission19.csv", index=False)